In [2]:
import tensorflow

In [3]:
### from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

In [4]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

In [5]:
# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

In [6]:
# Load the CIFAR10 data.
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#print(type(x_train))

In [7]:
import csv
import cv2
from tqdm import tqdm
DIR = './input/coic3/train'
x,y = [],[]

def make_train_label(DIR,x,y):
    for img in tqdm(os.listdir(DIR)):
        path = os.path.join(DIR,img)
        pic = cv2.imread(path,cv2.IMREAD_COLOR)
        x.append(np.array(pic))
        #--
        with open('./input/coic3/train_labels.csv', newline='') as csvfile:
            rows = csv.reader(csvfile)
            #print(dict(rows))
            name=img[:len(img)-4]
            y.append(dict(rows)[name])
make_train_label(DIR,x,y)

100%|██████████| 47500/47500 [12:27<00:00, 63.51it/s]


In [8]:
from sklearn.model_selection import train_test_split
x_train=np.array(x).astype('float32')/255
y_train=np.array(y).astype('int')
#y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
input_shape=x_train.shape[1:]
print(x_train.shape,y_train.shape)

x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2,random_state=42)
print(y_train.shape,y_test.shape)

(47500, 32, 32, 3) (47500,)
(38000,) (9500,)


In [9]:
# Input imashape dimensions.
#input_shape = x_train.shape[1:]

# Normalize data.
#x_train = x_train.astype('float32') / 255
#x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
#if subtract_pixel_mean:
#    x_train_mean = np.mean(x_train, axis=0)
#    x_train -= x_train_mean
#    x_test -= x_train_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
#y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

In [10]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [11]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [12]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
#model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks,
                       use_multiprocessing=True )

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Learning rate:  0.001
ResNet20v1
Using real-time data augmentation.
Learning rate:  0.001
Epoch 1/200
1188/1188 [==============================] - 132s 111ms/step - loss: 1.6765 - accuracy: 0.4430 - val_loss: 1.5415 - val_accuracy: 0.5170
Learning rate:  0.001
Epoch 2/200
1188/1188 [==============================] - 129s 109ms/step - loss: 1.3020 - accuracy: 0.5874 - val_loss: 1.2024 - val_accuracy: 0.6217
Learning rate:  0.001
Epoch 3/200
1188/1188 [==============================] - 127s 107ms/step - loss: 1.1318 - accuracy: 0.6536 - val_loss: 1.1917 - val_accuracy: 0.6478
Learning rate:  0.001
Epoch 4/200
1188/1188 [==============================] - 127s 107ms/step - loss: 1.0290 - accuracy: 0.6949 - val_loss: 1.0308 - val_accuracy: 0.7014
Learning rate:  0.001
Epoch 5/200
1188/1188 [==============================] - 128s 107ms/step - loss: 0.9599 - accuracy: 0.7225 - val_loss: 1.0713 - val_accuracy: 0.6921
Learning rate:  0.001
Epoch 6/200
1188/1188 [==============================] 

Process Keras_worker_ForkPoolWorker-400:
Process Keras_worker_ForkPoolWorker-397:
Process Keras_worker_ForkPoolWorker-398:
Process Keras_worker_ForkPoolWorker-399:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiproces

KeyboardInterrupt: 

In [27]:
DIR = './input/coic3/finaltest'

tmp1 = []

#def make_test(DIR,x):
#    for img in tqdm(os.listdir(DIR)):
#        path = os.path.join(DIR,img)
#        pic = cv2.imread(path,cv2.IMREAD_COLOR)
#        x.append(np.array(pic))
#make_test(DIR,tmp1)
for i in range(1,2501):
    path='./input/coic3/finaltest/test ({}).png'.format(i)
    pic=cv2.imread(path,cv2.IMREAD_COLOR)
    tmp1.append(np.array(pic))


In [28]:
print(os.listdir(DIR) [0:5])

['test (1207).png', 'test (1438).png', 'test (1888).png', 'test (1934).png', 'test (2238).png']


In [29]:
x_finaltest=np.array(tmp1).astype('float32')/255
print(x_finaltest.shape)

(2500, 32, 32, 3)


In [30]:
before_ans = model.predict(x_finaltest)

In [45]:
print(before_ans)
ans=[]
for i,a in enumerate(before_ans):
    m=[i+1,a.argmax()]
    ans.append(m)

[[2.0064214e-04 1.4541136e-07 9.6614623e-01 ... 1.4700645e-04
  4.2625126e-07 2.2551089e-07]
 [3.8702881e-05 1.2374438e-04 1.2597941e-06 ... 9.0635140e-06
  3.0759026e-07 9.9977404e-01]
 [7.1896958e-01 1.3126686e-01 2.1551435e-03 ... 7.0622722e-03
  2.7920471e-03 4.2863194e-02]
 ...
 [2.3215662e-01 8.5865483e-03 2.9215624e-04 ... 2.3887092e-03
  1.9886075e-02 7.3104960e-01]
 [1.6475153e-10 9.9999630e-01 1.6235112e-10 ... 1.8359129e-12
  1.2730927e-08 3.6833878e-06]
 [9.1837317e-01 5.5312488e-02 1.5871937e-04 ... 1.2033052e-04
  1.7711864e-03 8.8775381e-03]]


In [46]:
print(ans)

[[1, 2], [2, 9], [3, 0], [4, 4], [5, 9], [6, 5], [7, 8], [8, 5], [9, 5], [10, 3], [11, 4], [12, 9], [13, 9], [14, 6], [15, 2], [16, 9], [17, 7], [18, 4], [19, 3], [20, 1], [21, 0], [22, 1], [23, 7], [24, 2], [25, 5], [26, 0], [27, 6], [28, 8], [29, 2], [30, 1], [31, 4], [32, 8], [33, 1], [34, 2], [35, 3], [36, 1], [37, 6], [38, 0], [39, 4], [40, 3], [41, 1], [42, 3], [43, 1], [44, 8], [45, 5], [46, 6], [47, 6], [48, 3], [49, 2], [50, 1], [51, 0], [52, 7], [53, 6], [54, 3], [55, 3], [56, 5], [57, 7], [58, 0], [59, 7], [60, 7], [61, 0], [62, 8], [63, 8], [64, 2], [65, 1], [66, 2], [67, 1], [68, 6], [69, 1], [70, 9], [71, 4], [72, 4], [73, 7], [74, 4], [75, 9], [76, 7], [77, 4], [78, 9], [79, 6], [80, 0], [81, 6], [82, 0], [83, 1], [84, 9], [85, 2], [86, 0], [87, 7], [88, 2], [89, 1], [90, 2], [91, 7], [92, 3], [93, 9], [94, 2], [95, 2], [96, 6], [97, 8], [98, 8], [99, 7], [100, 0], [101, 6], [102, 2], [103, 5], [104, 9], [105, 7], [106, 4], [107, 1], [108, 7], [109, 8], [110, 8], [111, 8

In [47]:
import pandas as pd
sub = pd.DataFrame(ans)
sub = sub.rename(index=str, columns={0: "ID", 1: "Class"})
sub.to_csv('submission3.csv', index=False)

In [48]:
print(pd.DataFrame(ans))

         0  1
0        1  2
1        2  9
2        3  0
3        4  4
4        5  9
5        6  5
6        7  8
7        8  5
8        9  5
9       10  3
10      11  4
11      12  9
12      13  9
13      14  6
14      15  2
15      16  9
16      17  7
17      18  4
18      19  3
19      20  1
20      21  0
21      22  1
22      23  7
23      24  2
24      25  5
25      26  0
26      27  6
27      28  8
28      29  2
29      30  1
...    ... ..
2470  2471  3
2471  2472  9
2472  2473  2
2473  2474  1
2474  2475  4
2475  2476  4
2476  2477  8
2477  2478  1
2478  2479  7
2479  2480  3
2480  2481  5
2481  2482  2
2482  2483  3
2483  2484  2
2484  2485  4
2485  2486  8
2486  2487  7
2487  2488  2
2488  2489  5
2489  2490  1
2490  2491  4
2491  2492  2
2492  2493  0
2493  2494  1
2494  2495  0
2495  2496  0
2496  2497  3
2497  2498  9
2498  2499  1
2499  2500  0

[2500 rows x 2 columns]
